https://hesperia.gsfc.nasa.gov/rhessi3/data-access/rhessi-data/flare-list/index.html

http://sprg.ssl.berkeley.edu/~jimm/hessi/hsi_flare_list.html

http://sprg.ssl.berkeley.edu/%7Etohban/browser/?show=grth1+qlpcr+qlpds+qlpg9+qlifs+qli01+qli02+qli03+qli04+qli05+qli06+qlids&date=20020212&time=213530&bar=1
GOES classes: https://svs.gsfc.nasa.gov/10109

In [ ]:
import numpy as np
import scipp as sc

In [ ]:
da = sc.io.open_hdf5('data/hessi_flares.h5')

In [ ]:
da

In [ ]:
da.masks['non_solar'] = da.attrs.pop('non_solar')

In [ ]:
da.masks['unknown_position'] = (da.coords['x'] == 0*sc.Unit('asec')) | (da.coords['y'] == 0*sc.Unit('asec'))

In [ ]:
# manual edges to remove outliers
spatial = sc.bin(da,
                 edges=[
                     sc.linspace('y', -1200, 1200, 100, unit='asec'),
                     sc.linspace('x', -1200, 1200, 100, unit='asec')
                 ])

In [ ]:
fig = spatial.bins.sum().plot(aspect='equal', norm='log')
fig.ax.set_xlabel('x [asec]')
fig.ax.set_ylabel('y [asec]')
fig

Still lots of outliers -> remove

In [ ]:
# n bins such that bins are square
spatial = sc.bin(da,
                 edges=[
                     sc.linspace('y', -600, 600, 90, unit='asec'),
                     sc.linspace('x', -1000, 1000, 150, unit='asec')
                 ])

In [ ]:
fig = spatial.bins.sum().plot(aspect='equal', norm='log')
fig.ax.set_xlabel('x [asec]')
fig.ax.set_ylabel('y [asec]')
fig

Much better, now remove properly from `da`

In [ ]:
da = da[(abs(da.coords['x']) <= 1000*sc.Unit('asec')) & (abs(da.coords['y']) <= 600*sc.Unit('asec'))]

Included n xrays, look at just the number of flares

In [ ]:
weightless = sc.ones_like(da)

In [ ]:
spatial_weightless = sc.bin(weightless,
                 edges=[
                     sc.linspace('y', -600, 600, 90, unit='asec'),
                     sc.linspace('x', -1000, 1000, 150, unit='asec')
                 ])

In [ ]:
fig = spatial_weightless.bins.sum().plot(aspect='equal', norm='log')
fig.ax.set_xlabel('x [asec]')
fig.ax.set_ylabel('y [asec]')
fig

## Compare qualities

There are 7 quality levels

In [ ]:
np.unique(weightless.attrs['quality'].values)

with diminishing numbers of events

In [ ]:
weightless.groupby('quality').sum('event').plot(norm='log')

Bin by time first to see distribution of flares for all qualities

In [ ]:
min_time = da.coords['time'].min().value
max_time = da.coords['time'].max().value
step = (max_time - min_time) / 200
by_time = sc.bin(weightless, edges=[sc.arange('time', min_time, max_time, step, unit=da.coords['time'].unit)])

In [ ]:
by_time.bins.sum().plot()

Also bin by quality to split them into separate plots

In [ ]:
by_time_and_qual = sc.bin(by_time, groups=[sc.arange('quality', 1, 8, unit=None)])

In [ ]:
lines = {
    f"quality={by_time_and_qual['quality', i].attrs['quality'].value}":
    by_time_and_qual['quality', i].bins.sum()
    for i in range(by_time_and_qual.sizes['quality'])
}
sc.plot(lines, ylabel='n flares')

## Compare energies

Limit to Q1 and Q2

In [ ]:
best_quality = by_time_and_qual['quality', sc.index(1):sc.index(3)]

In [ ]:
min_energies = sc.array(dims=['min_energy'], 
                        values=np.unique(weightless.attrs['min_energy'].values),
                        unit=weightless.attrs['min_energy'].unit)
min_energies

N flares also diminishes but not as cleanly

In [ ]:
weightless.groupby('min_energy').sum('event').plot(norm='log')

Group by energy. Drop lowest because those events are not confirmed flares

In [ ]:
by_time_and_energy = sc.bin(by_time, groups=[min_energies[1:]])

In [ ]:
lines = {
    f"min_energy={by_time_and_energy['min_energy', i].attrs['min_energy'].value}":
    by_time_and_energy['min_energy', i].bins.sum()
    for i in range(by_time_and_energy.sizes['min_energy'])
}
sc.plot(lines, ylabel='n flares', norm='log')

## Durations

In [ ]:
durations = da.copy()
durations.data = durations.coords.pop('duration')
durations

In [ ]:
durations = durations[(durations.attrs['min_energy'] >= 6.0 * sc.Unit('keV')) &
                      (durations.attrs['quality'] < sc.index(3))]

In [ ]:
min_time = durations.coords['time'].min().value
max_time = durations.coords['time'].max().value
step = (max_time - min_time) / 200
durations_by_time = sc.bin(durations, edges=[sc.arange('time', min_time, max_time, step, unit=da.coords['time'].unit)])
durations_by_time.name = 'durations'

In [ ]:
durations_by_time.bins.mean().plot()